# Snowpark Dataframe Transformations Using Joins

The Snowpark API supports joins on DataFrame Objects.

---


### Create a Session
---

Create a Snowpark Session by passing in the connection properties file created in the [first lab exercise](../A-Dataframes/01-Sessions.ipynb).

In [ ]:
import com.snowflake.snowpark._
import com.snowflake.snowpark.functions._
import com.snowflake.snowpark.types._

// Set connection properties file variable
val pwd = sys.env.get("PWD").fold("")(_.toString)
val filename = s"$pwd/de_snowpark/connect.properties"

val session = Session.builder.configFile(s"$filename").create

---
### Create a DataFrame

Create a DataFrame with the following:

* Query the ONTIME_REPORTING table.
* Return the rows with YEAR = 2019
* Select the YEAR, ORIGIN, DEST, and OP_CARRIER columns.


In [ ]:
val routes2019DF =  session.table("raw.ONTIME_REPORTING")
                            .select(col("YEAR"),
                                       col("ORIGIN"), 
                                       col("DEST"),
                                       col("OP_CARRIER"))
                             .filter(col("YEAR") === 2019)
                                    

---

Create a DataFrame as a lookup to associate the carrier code to the carrier name.

In [ ]:
val carrierLookupDF = session.createDataFrame( Seq(("AA","American Airlines Inc."), 
                                         ("G4","Allegiant Air"),
                                         ("EV","ExpressJet Airlines LLC"),
                                         ("UA","United Air Lines Inc."),
                                         ("NK","Spirit Air Lines"),
                                         ("OO","SkyWest Airlines Inc."),
                                         ("HA","Hawaiian Airlines Inc."),
                                         ("OH","PSA Airlines Inc."),
                                         ("YX","Republic Airline"),
                                         ("AS","Alaska Airlines Inc."),
                                         ("9E","Endeavor Air Inc."),
                                         ("FDN","SnowBear Air "),   
                                        )).toDF("CODE", "DESCRIPTION")


---
Create a DataFrame that holds the list of distinct carriers.


In [ ]:
val carriers2019DF = routes2019DF.select(col("OP_CARRIER")).distinct

---
Create a DataFrame that joins the two DataFrames constructed above, on the columns containing the carrier code:

In [ ]:

val carrierNamesDF = carriers2019DF.join(carrierLookupDF, carriers2019DF.col("OP_CARRIER") === carrierLookupDF.col("CODE"))
                                   .select(col("OP_CARRIER"), col("DESCRIPTION"))
                                   .show()